In [ ]:
import numpy as np

In [ ]:
C      = 2.7e-12   # F
T      = 0.25/80e6 # s (55% of 80 MHz clock period -> in our case, this yields 9.9 ENOB at output)
thresh = 2**-8 / 2 # percent (half of 1 LSB (8 bits))
Vdd    = 1.5       # V

# From https://ihp-open-pdk-docs.readthedocs.io/en/latest/process_specs/02_process_control_params.html
Vth_n = 0.52   # V
Vth_p = 0.47   # V
L     = 130e-9 # m

# From https://github.com/IHP-GmbH/IHP-Open-PDK/blob/main/ihp-sg13g2/libs.tech/ngspice/models/sg13g2_moslv_parm.lib
mu_n    = 0.045582 # m^2/V*s
mu_p    = 0.017232 # m^2/V*s
tox_n   = 2.45e-9  # m
tox_p   = 2.65e-9  # m
epsox_n = 3.9
epsox_p = 3.9

In [ ]:
eps_0 = 8.854e-12               # F/m

Cox_n = epsox_n * eps_0 / tox_n # F/m^2
Cox_p = epsox_p * eps_0 / tox_p # F/m^2
print(f"Cox_n = {Cox_n:.3e} F/m^2")
print(f"Cox_p = {Cox_p:.3e} F/m^2")

In [ ]:
def find_Req(T, C, epsilon):
    # Vo(t) = Vdd * (1 - exp(-t / (C * Req)))
    # We want Vo(T) = Vdd * (1 - epsilon) (for simplicity, we just take Vdd)
    # => 1 - exp(-T / (C * Req)) = 1 - epsilon
    # => exp(-T / (C * Req)) = epsilon
    # => -T / (C * Req) = ln(epsilon)
    # => Req = -T / (C * ln(epsilon))
    return -T / (C * np.log(epsilon))

Req = find_Req(T, C, thresh)
print(f"Req = {Req:.2f} Ohm")

In [ ]:
def find_WL(Rind, mu, Cox, Vdd, Vth):
    # Id,sat = 1/2 * mu * Cox * (Vgs - Vth) ** 2
    # Rind approx Vdd / Id,sat
    # => Rind = Vdd / (1/2 * mu * Cox * (Vdd - Vth) ** 2)
    # => Rind = Vdd * 2 / (mu * Cox * (Vdd - Vth) ** 2)
    return 2 * Vdd / (Rind * mu * Cox * (Vdd - Vth) ** 2)

WL_n = find_WL(Req, mu_n, Cox_n, Vdd, Vth_n)
WL_p = find_WL(Req, mu_p, Cox_p, Vdd, Vth_p)
print(f"W/L_n = {WL_n:.2f}")
print(f"W/L_p = {WL_p:.2f}")

W_n = WL_n * L
W_p = WL_p * L
print(f"W_n = {W_n*1e6:.2f} um")
print(f"W_p = {W_p*1e6:.2f} um")